In [18]:
from pathlib import Path
import os, shutil
import pandas as pd
from dotenv import load_dotenv

CWD = Path.cwd()
PROJECT_ROOT = CWD.parent if CWD.name == "notebooks" else CWD


nb_env = CWD / ".env"
root_env = PROJECT_ROOT / ".env"
if CWD.name == "notebooks" and nb_env.exists():
    # only move if root doesn't already have one
    if not root_env.exists():
        shutil.move(str(nb_env), str(root_env))
        print(f"Moved .env -> {root_env}")
    else:
        print(f".env already exists at {root_env}; leaving notebooks/.env alone")

load_dotenv(root_env, override=True)
ALPHA_KEY = os.getenv("ALPHAVANTAGE_API_KEY")

DATA_RAW = PROJECT_ROOT / "data" / "raw"
DATA_RAW.mkdir(parents=True, exist_ok=True)

nb_raw = CWD / "data" / "raw"
moved = []
if CWD.name == "notebooks" and nb_raw.exists():
    for p in nb_raw.glob("*.csv"):
        target = DATA_RAW / p.name
        shutil.move(str(p), str(target))
        moved.append(target.name)

print("PROJECT_ROOT:", PROJECT_ROOT)
print("Using .env:", root_env, "| Exists:", root_env.exists(), "| Has ALPHA key?", bool(ALPHA_KEY))
print("DATA_RAW:", DATA_RAW)
print("Moved from notebooks/data/raw -> project/data/raw:", moved if moved else "None")


PROJECT_ROOT: /Users/wangyuhan/bootcamp_Serena_Wang/project
Using .env: /Users/wangyuhan/bootcamp_Serena_Wang/project/.env | Exists: True | Has ALPHA key? True
DATA_RAW: /Users/wangyuhan/bootcamp_Serena_Wang/project/data/raw
Moved from notebooks/data/raw -> project/data/raw: ['api_source-yfinance_symbol-nvda_2025-08-17T19-49-40.csv']


In [20]:

import pandas as pd, requests

try:
    validate_df
except NameError:
    def validate_df(df: pd.DataFrame, *, required_cols, dtypes_map=None, min_rows=10, allow_na_frac=None):
        missing = [c for c in required_cols if c not in df.columns]
        if missing: raise ValueError(f"Missing required columns: {missing}")
        if dtypes_map:
            for col, kind in dtypes_map.items():
                if col in df.columns:
                    if str(kind).startswith("datetime"):
                        df[col] = pd.to_datetime(df[col], errors="coerce")
                    elif str(kind).startswith("float"):
                        df[col] = pd.to_numeric(df[col], errors="coerce")
        if len(df) < min_rows:
            raise ValueError(f"Too few rows: {len(df)} < {min_rows}")
        return [f"Shape: {df.shape}", f"NA counts: {df[required_cols].isna().sum().to_dict()}"]

try:
    safe_filename
except NameError:
    def safe_filename(prefix: str, meta: dict, ext="csv"):
        stamp = pd.Timestamp.utcnow().strftime("%Y-%m-%dT%H-%M-%S")
        parts = [prefix] + [f"{k}-{str(v).lower()}" for k, v in meta.items()]
        return "_".join(parts) + f"_{stamp}.{ext}"

assert 'ALPHA_KEY' in globals() and 'DATA_RAW' in globals(), "Run the setup cell first to set ALPHA_KEY and DATA_RAW."

SYMBOL = "NVDA" 

def fetch_alpha_vantage_adj(symbol: str, api_key: str):
    url = "https://www.alphavantage.co/query"
    params = {
        "function": "TIME_SERIES_DAILY_ADJUSTED",
        "symbol": symbol,
        "outputsize": "compact",
        "apikey": api_key,
        "datatype": "json",
    }
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    js = r.json()
    # throttle/errors → return None and the reason
    err = js.get("Error Message") or js.get("Information") or js.get("Note")
    if err:
        return None, err
    key = next((k for k in js.keys() if "Time Series" in k), None)
    if not key:
        return None, f"Unexpected keys: {list(js.keys())}"
    series = js[key]
    df = (
        pd.DataFrame(series).T
        .rename_axis("date")
        .reset_index()
        .loc[:, ["date", "5. adjusted close"]]
        .rename(columns={"5. adjusted close": "adj_close"})
    )
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df["adj_close"] = pd.to_numeric(df["adj_close"], errors="coerce")
    return df, None

def fetch_yf(symbol: str):
    import yfinance as yf
    ydf = yf.download(symbol, period="6mo", interval="1d",
                      auto_adjust=False, group_by="column", progress=False)
    if ydf.empty:
        raise RuntimeError(f"yfinance returned empty data for {symbol!r}")
    ydf = ydf.reset_index()
    # flatten if MultiIndex
    if isinstance(ydf.columns, pd.MultiIndex):
        ydf.columns = [" ".join([str(x) for x in tup if x]).strip() for tup in ydf.columns.to_list()]
    cols = [str(c) for c in ydf.columns]
    date_col = "Date" if "Date" in cols else ("Datetime" if "Datetime" in cols else None)
    if not date_col:
        raise KeyError(f"No Date/Datetime column in: {cols}")
    price_candidates = [c for c in cols if c.replace(" ", "").lower().startswith("adjclose")]
    if not price_candidates:
        price_candidates = [c for c in cols if c.replace(" ", "").lower().startswith("close")]
    if not price_candidates:
        raise KeyError(f"Neither 'Adj Close*' nor 'Close*' in: {cols}")
    price_col = price_candidates[0]
    df = ydf[[date_col, price_col]].rename(columns={date_col: "date", price_col: "adj_close"})
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df["adj_close"] = pd.to_numeric(df["adj_close"], errors="coerce")
    return df

source = None
if ALPHA_KEY:
    df_api, err = fetch_alpha_vantage_adj(SYMBOL, ALPHA_KEY)
    if df_api is None:
        print("Alpha Vantage unavailable →", err)
        print("Falling back to yfinance…")
        df_api = fetch_yf(SYMBOL)
        source = "yfinance"
    else:
        source = "alpha"
else:
    df_api = fetch_yf(SYMBOL)
    source = "yfinance"


df_api = df_api.sort_values("date").reset_index(drop=True)
msgs = validate_df(df_api, required_cols=["date", "adj_close"],
                   dtypes_map={"date": "datetime64[ns]", "adj_close": "float"})
print("\n".join(msgs))

fname = safe_filename("api", {"source": source, "symbol": SYMBOL})
out_path = DATA_RAW / fname
df_api.to_csv(out_path, index=False)
print("Saved:", out_path)


Alpha Vantage unavailable → Thank you for using Alpha Vantage! This is a premium endpoint. You may subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly unlock all premium endpoints
Falling back to yfinance…
Shape: (125, 2)
NA counts: {'date': 0, 'adj_close': 0}
Saved: /Users/wangyuhan/bootcamp_Serena_Wang/project/data/raw/api_source-yfinance_symbol-nvda_2025-08-17T20-00-42.csv


In [25]:
# Scrape Table
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from pathlib import Path

try:
    DATA_RAW
except NameError:
    PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
    DATA_RAW = PROJECT_ROOT / "data" / "raw"
    DATA_RAW.mkdir(parents=True, exist_ok=True)

try:
    validate_df
except NameError:
    def validate_df(df: pd.DataFrame, *, required_cols, dtypes_map=None, min_rows=5, allow_na_frac=None):
        missing = [c for c in required_cols if c not in df.columns]
        if missing: raise ValueError(f"Missing required columns: {missing}")
        if dtypes_map:
            for col, kind in dtypes_map.items():
                if col in df.columns:
                    if str(kind).startswith("datetime"):
                        df[col] = pd.to_datetime(df[col], errors="coerce")
                    elif str(kind).startswith("float"):
                        df[col] = pd.to_numeric(df[col], errors="coerce")
        if len(df) < min_rows:
            raise ValueError(f"Too few rows: {len(df)} < {min_rows}")
        return [f"Shape: {df.shape}", f"NA counts: {df[required_cols].isna().sum().to_dict()}"]

try:
    safe_filename
except NameError:
    def safe_filename(prefix: str, meta: dict, ext="csv"):
        stamp = pd.Timestamp.utcnow().strftime("%Y-%m-%dT%H-%M-%S")
        parts = [prefix] + [f"{k}-{str(v).lower()}" for k, v in meta.items()]
        return "_".join(parts) + f"_{stamp}.{ext}"

def clean_text(s: str) -> str:
    if s is None: return ""
    s = re.sub(r"\s*\[[^\]]*\]\s*", " ", s)  # remove [citation] brackets
    s = s.replace("\u00a0", " ").replace("\u2009", " ")
    s = re.sub(r"\s+", " ", s).strip()
    return s

def parse_wikitable(tbl):
    """Parse a single wikitable into a DataFrame and return (df, caption_text)."""
    caption = tbl.find("caption")
    cap_text = clean_text(caption.get_text(" ", strip=True)) if caption else ""

    # pick the first row that actually has THs as header
    header_row = None
    for tr in tbl.find_all("tr"):
        if tr.find("th"):
            header_row = tr
            break
    if header_row is None:
        return None

    headers = [clean_text(th.get_text(" ", strip=True)) for th in header_row.find_all(["th","td"])]
    rows = []
    for tr in header_row.find_next_siblings("tr"):
        cells = [clean_text(td.get_text(" ", strip=True)) for td in tr.find_all(["td","th"])]
        if not cells:
            continue
        # normalize length to header
        if len(cells) < len(headers):
            cells += [""] * (len(headers) - len(cells))
        elif len(cells) > len(headers):
            cells = cells[:len(headers)]
        rows.append(cells)
    if not rows:
        return None

    df = pd.DataFrame(rows, columns=headers)
    return df, cap_text

URL = "https://en.wikipedia.org/wiki/Nvidia"
headers = {"User-Agent": "AFE-Course-Notebook/1.0 (contact: instructor@example.edu)"}


resp = requests.get(URL, headers=headers, timeout=30)
resp.raise_for_status()
soup = BeautifulSoup(resp.text, "html.parser")

tables = soup.select("table.wikitable")
candidates = []
for tbl in tables:
    parsed = parse_wikitable(tbl)
    if parsed:
        df, cap = parsed
        candidates.append((df, cap, len(df)))

if not candidates:
    raise RuntimeError("No parseable wikitables found on the page.")

print("Found wikitable candidates (index, rows, caption snippet):")
for i, (df, cap, n) in enumerate(candidates):
    print(f"  {i}: rows={n:>3} | {cap[:60]}")


preferred = [i for i,(df,cap,n) in enumerate(candidates) if "acquisition" in cap.lower()]
if preferred:
    idx = preferred[0]
else:
    big = [i for i,(df,cap,n) in enumerate(candidates) if n >= 5]
    idx = max(big, key=lambda i: candidates[i][2]) if big else max(range(len(candidates)), key=lambda i: candidates[i][2])

df_scrape, caption, nrows = candidates[idx]
print(f"\nUsing table index {idx} | rows={nrows} | caption: {caption or '(no caption)'}")


for c in df_scrape.columns:
    s = df_scrape[c].astype(str).str.replace(",", "", regex=False).str.replace("\u2009","",regex=False)
    # try to detect numerics; keep as text if too few convert
    nums = pd.to_numeric(s.str.replace(r"[^0-9\.\-]", "", regex=True), errors="coerce")
    if nums.notna().mean() >= 0.5:
        df_scrape[c] = nums


required = list(df_scrape.columns[:2]) if df_scrape.shape[1] >= 2 else list(df_scrape.columns)
msgs = validate_df(df_scrape, required_cols=required, min_rows=5)
print("\n".join(msgs))


slug = re.sub(r"[^a-z0-9]+", "-", (caption.lower() or "wikitable")).strip("-")
fname = safe_filename("scrape", {"source":"wikipedia","page":"nvidia","table": slug})
out_path = DATA_RAW / fname
df_scrape.to_csv(out_path, index=False)
print("Saved:", out_path)


print("\nPreview:")
print(df_scrape.head(10))


Found wikitable candidates (index, rows, caption snippet):
  0: rows=  2 | Sales by business unit (2023)
  1: rows=  4 | Sales by region (2023)
  2: rows= 10 | 10-year financials (2016–2025)

Using table index 2 | rows=10 | caption: 10-year financials (2016–2025)
Shape: (10, 4)
NA counts: {'Year': 0, 'Revenue (mn. US$)': 0}
Saved: /Users/wangyuhan/bootcamp_Serena_Wang/project/data/raw/scrape_source-wikipedia_page-nvidia_table-10-year-financials-2016-2025_2025-08-17T23-08-14.csv

Preview:
   Year  Revenue (mn. US$)  Net income (mn. US$)  Employees
0  2016               5010                   614       9227
1  2017               6910                  1666      10299
2  2018               9714                  3047      11528
3  2019              11716                  4141      13277
4  2020              10918                  2796      13775
5  2021              16675                  4332      18975
6  2022              26914                  9752      22473
7  2023              26974 

# Documentation
**API Source**
- Alpha Vantage `TIME_SERIES_DAILY_ADJUSTED` (fallback: yfinance)
- URL: https://www.alphavantage.co/query
- Params: `function=TIME_SERIES_DAILY_ADJUSTED`, `symbol=${SYMBOL}`, `outputsize=compact`, `datatype=json`, `apikey=$ALPHAVANTAGE_API_KEY`

**Scrape Source**
- Wikipedia — Nvidia: https://en.wikipedia.org/wiki/Nvidia
- Parsed the first substantial `table.wikitable` (prefer “Acquisitions” if present).

**Validation**
- API: required cols `date, adj_close`; coerced `date→datetime`, `adj_close→float`; ≥10 rows; NA counts printed.
- Scrape: ≥2 cols, ≥5 rows; light numeric coercion; NA counts printed.


In [ ]:
 **Assumptions & Risks**
- Alpha Vantage throttling/premium → fallback to yfinance.
- Wikipedia tables may change; parsing uses simple heuristics.